# Computer Vision 1: Assignment 1

Group Number 43

Marek Drwal

## setup

In [4]:
import imageio
import numpy as np
import skimage as ski

## task 1

### data

Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009, https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

In [5]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [6]:
data_dir = "C:/Users/admin/Documents/cifar-10-python/cifar-10-batches-py"

In [8]:
label_names = unpickle(data_dir + '/batches.meta')

### batch 1

In [7]:
data_batch_1 = unpickle(data_dir + '/data_batch_1')

In [9]:
# extracted_classes = data_batch_1[b'data'][data_batch_1[b'labels'] in label_names[b'label_names'][1, 4, 8]]
extracted_classes = data_batch_1[b'data'][data_batch_1[b'labels'] in [1,4,8]]

In [10]:
data_batch_1[b'data'][np.isin(np.array(data_batch_1[b'labels']), [1,4,8])]

array([[ 28,  37,  38, ...,  28,  37,  46],
       [170, 168, 177, ...,  82,  78,  80],
       [159, 150, 153, ...,  14,  17,  19],
       ...,
       [116, 120, 126, ...,  82,  84,  81],
       [ 71,  60,  74, ...,  68,  69,  68],
       [250, 254, 211, ..., 215, 255, 254]], dtype=uint8)

#### first 30 in classes

In [ ]:
selected_classes = [1,4,8]

In [ ]:
data_batch_1_labels = np.array(data_batch_1[b'labels'])
extracted_top_30_list = []

for class_lab in selected_classes:
    first30 = data_batch_1[b'data'][data_batch_1_labels == class_lab][:30,]
    print(len(first30))
    extracted_top_30_list.append(first30)

extracted_top_30_list

30
30
30


[array([[170, 168, 177, ...,  82,  78,  80],
        [159, 150, 153, ...,  14,  17,  19],
        [ 50,  51,  42, ..., 166, 166, 162],
        ...,
        [214, 212, 214, ...,  58,  60,  61],
        [ 48, 128, 156, ...,  20,  19,  30],
        [ 47,  57,  70, ...,   2,   2,   3]], dtype=uint8),
 array([[ 28,  37,  38, ...,  28,  37,  46],
        [ 53,  54,  56, ...,  39,  46,  41],
        [153, 174, 155, ..., 217, 217, 219],
        ...,
        [ 45,  51,  77, ..., 116, 113, 134],
        [ 30,  27,  29, ...,  64,  62,  67],
        [ 89,  85,  81, ...,  75,  60,  48]], dtype=uint8),
 array([[134, 131, 128, ..., 136, 137, 138],
        [ 76,  73,  69, ..., 127, 127, 130],
        [151, 150, 151, ..., 102, 102, 103],
        ...,
        [ 58,  51,  50, ...,  70,  63,  57],
        [196, 203, 212, ...,  94, 100, 106],
        [ 61,  62,  62, ...,  92,  90,  91]], dtype=uint8)]

In [12]:
extracted_top_30_list

[array([[170, 168, 177, ...,  82,  78,  80],
        [159, 150, 153, ...,  14,  17,  19],
        [ 50,  51,  42, ..., 166, 166, 162],
        ...,
        [214, 212, 214, ...,  58,  60,  61],
        [ 48, 128, 156, ...,  20,  19,  30],
        [ 47,  57,  70, ...,   2,   2,   3]], dtype=uint8),
 array([[ 28,  37,  38, ...,  28,  37,  46],
        [ 53,  54,  56, ...,  39,  46,  41],
        [153, 174, 155, ..., 217, 217, 219],
        ...,
        [ 45,  51,  77, ..., 116, 113, 134],
        [ 30,  27,  29, ...,  64,  62,  67],
        [ 89,  85,  81, ...,  75,  60,  48]], dtype=uint8),
 array([[134, 131, 128, ..., 136, 137, 138],
        [ 76,  73,  69, ..., 127, 127, 130],
        [151, 150, 151, ..., 102, 102, 103],
        ...,
        [ 58,  51,  50, ...,  70,  63,  57],
        [196, 203, 212, ...,  94, 100, 106],
        [ 61,  62,  62, ...,  92,  90,  91]], dtype=uint8)]

In [13]:
len(extracted_top_30_list)

3

In [20]:
print(extracted_top_30_list[0].size/30)
extracted_top_30_list[0][1].size

3072.0


3072

#### histograms

In [24]:
hists = []
bin_size = 5
no_bins = 51

for img_class in extracted_top_30_list:
    for img in img_class:
        red = img[:1024].reshape((32, 32)).astype(np.float64)
        green = img[1024:2048].reshape((32, 32)).astype(np.float64)
        blue = img[2048:].reshape((32, 32)).astype(np.float64)
        gray = red + green + blue
        gray /= 3
        hh = np.zeros((no_bins+1)) # extra one for 255
        for px in gray:
            hh[px.astype(np.int8)//bin_size] += 1
        hh[no_bins-1] += hh[-1] # adjust the end
        if (hh[-1] > 0):
            print(hh[-1])
        hists.append(hh)
        print(hh)

[ 5.  9.  6. 14.  9. 11. 11.  8.  8.  8.  6. 11. 13. 14. 17. 17. 22. 17.
 17. 16. 17. 14. 11. 15. 11.  9.  7.  9.  6.  4.  6.  4.  4.  6. 10.  8.
 13. 11. 12.  7.  4.  5.  5.  5.  4.  4.  4.  2.  2.  2.  1.  0.]
[ 0.  0.  3.  8.  5.  8. 13. 14. 16. 12. 18. 17. 16. 20. 24. 19. 22. 19.
 14. 14. 16. 16. 20. 20. 22. 16. 14. 25. 17. 12. 11. 16. 12. 11.  5.  8.
  3.  1.  2.  2.  1.  2.  0.  2.  0.  1.  0.  1.  2.  0.  0.  0.]
1.0
[12. 15. 14. 12. 13. 16. 16. 14. 14. 16. 19. 15. 16. 22. 18. 15. 20. 19.
 16. 15.  6. 13. 13. 14. 16.  9.  7. 14. 10.  5. 14. 15. 16. 16. 13. 17.
 10. 13.  6.  4.  6.  3.  3.  4.  4.  4.  3.  3.  2.  1.  2.  1.]
1.0
[ 0.  0.  2.  3.  5. 10. 13. 11. 14. 19. 16. 18. 25. 25. 20. 22. 19. 14.
 20. 14. 20. 26. 23. 18. 21. 12. 10. 17. 17. 17. 15.  9. 13. 12. 12.  7.
  8.  8.  9.  5. 10.  5.  5.  1.  2.  2.  2.  2.  1.  0.  2.  1.]
[ 5.  9.  7. 10. 13. 12.  9. 13. 15. 18. 19. 20. 17. 16. 16. 19. 21. 20.
 18. 18. 20. 16. 19. 18. 17. 14. 11. 15. 17. 14. 10.  9.  7.  9.  3.  6

In [ ]:
bin_size = 5
no_bins = 51

for i in range(no_bins):
    print(i, i*bin_size, (i+1)*bin_size) # left closed, right open

0 0 5
1 5 10
2 10 15
3 15 20
4 20 25
5 25 30
6 30 35
7 35 40
8 40 45
9 45 50
10 50 55
11 55 60
12 60 65
13 65 70
14 70 75
15 75 80
16 80 85
17 85 90
18 90 95
19 95 100
20 100 105
21 105 110
22 110 115
23 115 120
24 120 125
25 125 130
26 130 135
27 135 140
28 140 145
29 145 150
30 150 155
31 155 160
32 160 165
33 165 170
34 170 175
35 175 180
36 180 185
37 185 190
38 190 195
39 195 200
40 200 205
41 205 210
42 210 215
43 215 220
44 220 225
45 225 230
46 230 235
47 235 240
48 240 245
49 245 250
50 250 255


### test batch

In [ ]:
test_batch = unpickle(data_dir + '/test_batch')

#### first 10 in classes

In [ ]:
test_batch_labels = np.array(test_batch[b'labels'])
extracted_top_10_list = []

for class_lab in selected_classes:
    first10 = test_batch[b'data'][test_batch_labels == class_lab][:10,]
    print(len(first10))
    extracted_top_10_list.append(first10)

extracted_top_10_list

30
30
30


[array([[170, 168, 177, ...,  82,  78,  80],
        [159, 150, 153, ...,  14,  17,  19],
        [ 50,  51,  42, ..., 166, 166, 162],
        ...,
        [214, 212, 214, ...,  58,  60,  61],
        [ 48, 128, 156, ...,  20,  19,  30],
        [ 47,  57,  70, ...,   2,   2,   3]], dtype=uint8),
 array([[ 28,  37,  38, ...,  28,  37,  46],
        [ 53,  54,  56, ...,  39,  46,  41],
        [153, 174, 155, ..., 217, 217, 219],
        ...,
        [ 45,  51,  77, ..., 116, 113, 134],
        [ 30,  27,  29, ...,  64,  62,  67],
        [ 89,  85,  81, ...,  75,  60,  48]], dtype=uint8),
 array([[134, 131, 128, ..., 136, 137, 138],
        [ 76,  73,  69, ..., 127, 127, 130],
        [151, 150, 151, ..., 102, 102, 103],
        ...,
        [ 58,  51,  50, ...,  70,  63,  57],
        [196, 203, 212, ...,  94, 100, 106],
        [ 61,  62,  62, ...,  92,  90,  91]], dtype=uint8)]

#### histograms

In [ ]:
hists_test = []

for img_class in extracted_top_10_list:
    for img in img_class:
        red = img[:1024].reshape((32, 32)).astype(np.float64)
        green = img[1024:2048].reshape((32, 32)).astype(np.float64)
        blue = img[2048:].reshape((32, 32)).astype(np.float64)
        gray = red + green + blue
        gray /= 3
        hh = np.zeros((no_bins+1)) # extra one for 255
        for px in gray:
            hh[px.astype(np.int8)//bin_size] += 1
        hh[no_bins-1] += hh[-1] # adjust the end
        if (hh[-1] > 0):
            print(hh[-1])
        hists_test.append(hh)
        print(hh)

[ 5.  9.  6. 14.  9. 11. 11.  8.  8.  8.  6. 11. 13. 14. 17. 17. 22. 17.
 17. 16. 17. 14. 11. 15. 11.  9.  7.  9.  6.  4.  6.  4.  4.  6. 10.  8.
 13. 11. 12.  7.  4.  5.  5.  5.  4.  4.  4.  2.  2.  2.  1.  0.]
[ 0.  0.  3.  8.  5.  8. 13. 14. 16. 12. 18. 17. 16. 20. 24. 19. 22. 19.
 14. 14. 16. 16. 20. 20. 22. 16. 14. 25. 17. 12. 11. 16. 12. 11.  5.  8.
  3.  1.  2.  2.  1.  2.  0.  2.  0.  1.  0.  1.  2.  0.  0.  0.]
1.0
[12. 15. 14. 12. 13. 16. 16. 14. 14. 16. 19. 15. 16. 22. 18. 15. 20. 19.
 16. 15.  6. 13. 13. 14. 16.  9.  7. 14. 10.  5. 14. 15. 16. 16. 13. 17.
 10. 13.  6.  4.  6.  3.  3.  4.  4.  4.  3.  3.  2.  1.  2.  1.]
1.0
[ 0.  0.  2.  3.  5. 10. 13. 11. 14. 19. 16. 18. 25. 25. 20. 22. 19. 14.
 20. 14. 20. 26. 23. 18. 21. 12. 10. 17. 17. 17. 15.  9. 13. 12. 12.  7.
  8.  8.  9.  5. 10.  5.  5.  1.  2.  2.  2.  2.  1.  0.  2.  1.]
[ 5.  9.  7. 10. 13. 12.  9. 13. 15. 18. 19. 20. 17. 16. 16. 19. 21. 20.
 18. 18. 20. 16. 19. 18. 17. 14. 11. 15. 17. 14. 10.  9.  7.  9.  3.  6

In [ ]:
def L2_norm(h1, h2):
    return np.sqrt(np.sum(np.square(h1-h2)))